https://github.com/SeleniumHQ/docker-selenium

docker run -d -p 4444:4444 -p 7900:7900 --shm-size="2g" selenium/standalone-firefox:latest

Point your WebDriver tests to http://localhost:4444
To see what is happening inside the container, head to http://localhost:7900 (password is secret)


In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By
from re import sub as re_sub
from collections import namedtuple
from decimal import Decimal


In [2]:
options = FirefoxOptions()
driver = webdriver.Remote(
    command_executor='http://172.17.0.3:4444',
    options=options
)

In [3]:
driver.get('https://www.mikroe.com/click')
product_menu=driver.find_element(By.XPATH, "(//*[contains(@class, 'tree dynamized')])")
cb_menu=product_menu.find_element(By.PARTIAL_LINK_TEXT,"Click Boards")
cb_categories = cb_menu.find_elements(By.XPATH, './../ul/*')

In [4]:
items = []
Subcategory = namedtuple("Subcategory", "name href parent")

for category in cb_categories:
    _ = category.find_element(By.TAG_NAME, 'a').get_attribute("textContent")
    parent = re_sub(r"\([^()]*\)", "", _).strip()
    if parent in ["Click Bundles", "Click Shields", "Legacy"]:
        continue
    subcategories = category.find_elements(By.XPATH, 'ul/*')
    for subcategory in subcategories:
        _ = subcategory.find_element(By.TAG_NAME, 'a').get_attribute("textContent")
        name = re_sub(r"\([^()]*\)", "", _).strip()
        href = subcategory.find_element(By.TAG_NAME, 'a').get_attribute("href")
        
        items.append(Subcategory(name, href, parent))


In [8]:
Clickboard = namedtuple("Clickboard", "subcategory code name url description price")
clickboards = []

driver.get(str(items[1].href))
products=driver.find_elements(By.XPATH, "(//*[contains(@class, 'product-container')])")
for product in products:
    code = product.find_element(By.CSS_SELECTOR, "div.content_price span.product-reference").get_attribute("textContent")
    name = product.find_element(By.CLASS_NAME, "product-name").get_attribute("textContent")
    url = product.find_element(By.CLASS_NAME, "product-name").get_attribute("href")
    description = product.find_element(By.CLASS_NAME, "product-desc").get_attribute("textContent")
    price = product.find_element(By.CLASS_NAME, "me-product-price").get_attribute("textContent")
    price = Decimal(re_sub(r'[^\d.]', '', price))

    clickboards.append(Clickboard(str(items[1].name), code, name, url, description, price))

In [9]:
clickboards

[Clickboard(subcategory='GSM/LTE', code='MIKROE-1298', name='GSM Click', url='https://www.mikroe.com/gsm-click', description='GSM click is a compact and powerful GSM cellular network communication solution, featuring the GSM/GPRS Telit\xa0GL865-QUAD\xa0module.\xa0This module features a full set of options for the cellular networking and communication, such as the network status indication, jamming detection, embedded TCP/IP stack, including TCP, IP, UDP, SMTP, ICMP and FTP protocols,...', price=Decimal('49.00')),
 Clickboard(subcategory='GSM/LTE', code='MIKROE-1375', name='GSM2 Click', url='https://www.mikroe.com/gsm-2-click', description='GSM2 click\xa0is a compact quad-band GSM cellular network communication solution, featuring the GSM/GPRS Quectel M95.\xa0This module features a full set of options for the cellular networking and communication, such as the network status indication, jamming detection, embedded internet protocols, including TCP/IP, UDP, FTP, PPP, HTTP, SMTP, full\xa0G

In [ ]:
# things = driver.find_elements(By.XPATH, "//div[@class='af_pl_wrapper']")
# things = driver.find_elements(By.CSS_SELECTOR, "li.ajax_block_product")
# things = driver.find_elements(By.CSS_SELECTOR, "div.product-container.clearfix")
# things = driver.find_elements(By.XPATH, "(//*[contains(@class, 'product-container')])")
# x = things[0].find_element(By.CSS_SELECTOR, "p.product-desc")

In [10]:
driver.quit()